### Since we're running everything on Google's remote servers, we need to re-load all the software packages we'll use each time we open a notebook.

### Please run the following two cells to install all the necessary software. Note that it could take a couple of minutes.

In [2]:
#STEP 1: Enable the "Anaconda" package manager in this Google Colab Notebook:
!pip install -q condacolab
import condacolab
condacolab.install()

RuntimeError: This module must ONLY run as part of a Colab notebook!

In [ ]:
#STEP 2: Install the free SCIP optimization problem solver
!conda install conda-forge::scip

#STEP 3: Download and install my "PyomoTools" package.
#   This will install all other pieces of software we need.
!git clone https://github.com/NathanDavisBarrett/PyomoTools.git
%cd PyomoTools
!pip install .
%cd ..

#At the end of this cell, you'll get a popup window asking you to restart. Please click "cancel".

# Math Modeling With Pyomo

In [3]:
import pyomo.environ as pyo
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# SECTION 1: Getting to know Pyomo Objects (Etsy Shop Problem)

### Pyomo ConcreteModel

Syntax:

```__ModelName__ = pyo.ConcreteModel()```

In [4]:
model = pyo.ConcreteModel()

### Pyomo Sets

Syntax:

```__ModelName__.__SetName__ = pyo.Set(initialize=__SetContents__)```

In math, we have the following sets:

* $p \in \textbf{P}$: A set of all people
* $r \in \textbf{R}$: A set of all products

Here's how we'd write it in python:

In [5]:
model.peopleSet = pyo.Set(initialize=["Jack","Jill"])
model.productSet = pyo.Set(initialize=["Sculptures","Shirts"])

### Parameters

Not explicitly represented in Pyomo. Just use regular Python variables.

In math, we have the following parameters:
* $\alpha_{p,r}$
* $\tau$
* $\kappa$

Here's how we'd write it in python:

In [6]:
#The number of each product that can be produced by each person in an hour
#   Parameters can also be loaded from an Excel Sheet. We'll cover how to do that later.
alpha = {
    "Jack": {
        "Shirts": 5,
        "Sculptures": 2
    },
    "Jill": {
        "Shirts": 9,
        "Sculptures": 3
    }
}

#The number of hours each person can work in a day
tau = 8

#The number of shirts needed
kappa = 25

### Pyomo Variables

Syntax:

```__ModelName__.__VariableName__ = pyo.Var(__SetToDefineOver__,domain=__DomainYouWant__)```

In math, we have the following variables:

* $H_{p,r} \ \ \ \forall p \in \textbf{P}, r \in \textbf{R}$
* $N_r \ \ \ \forall r \in \textbf{R}$

Here's how we'd write it in python:

In [7]:
model.H = pyo.Var(model.peopleSet * model.productSet, domain=pyo.NonNegativeReals)
model.N = pyo.Var(model.productSet, domain=pyo.NonNegativeReals)

### Pyomo Constraints

Syntax:

```__ModelName__.__ConstraintName__ = pyo.Constraint(__SetToDefineOver__,rule=__ConstructingFunction__)```

In math, we have the following constraints:

$$0 \leq \sum_{r \in \textbf{R}} H_{p,r} \leq \tau^{DAY} \ \ \forall p \in \textbf{P}$$
$$N_{r} = \sum_{p \in \textbf{P}} \alpha_{p,r} H_{p,r}\ \ \ \forall r \in \textbf{R}$$
$$N_{Shirts} = \kappa$$

Here's how we'd write it in python:

In [8]:
# Note that 0 <= sum(H) <= tau is actually two constraints:
#   0 <= sum(H)
#   sum(H) <= tau
# In python, we need to write it as two separate constraints.
def WorkTimeLimit_1(model,p):
    return 0 <= sum(model.H[p,r] for r in model.productSet)

model.WorkTimeLimit_1 = pyo.Constraint(model.peopleSet, rule=WorkTimeLimit_1)


def WorkTimeLimit_2(model,p):
    return sum(model.H[p,r] for r in model.productSet) <= tau

model.WorkTimeLimit_2 = pyo.Constraint(model.peopleSet, rule=WorkTimeLimit_2)


def N_Definition(model,r):
    return model.N[r] == sum(alpha[p][r] * model.H[p,r] for p in model.peopleSet)

model.N_Definition = pyo.Constraint(model.productSet, rule=N_Definition)


def ShirtLimit(model):
    return model.N["Shirts"] == kappa

model.ShirtLimit = pyo.Constraint(rule=ShirtLimit)

### Objective

Syntax:

```__ModelName__.__ObjectiveName__ = pyo.Objective(expr=__YourExpression__,sense=__YourSense)```

Example Objective:

$$\max \sum_{r \in \textbf{R}} N_r$$

In [9]:
model.myObj = pyo.Objective(expr=sum(model.N[r] for r in model.productSet), sense=pyo.maximize)

### Solver

Syntax:

```__SolverObjectName__ = pyo.SolverFactory(__NameOfTheSolverYouWantToUse__)```

```__SolverObjectName__.solve(__ModelName__,...__OtherOptions__...)```

In [10]:
solver = pyo.SolverFactory("scip")

solver.solve(model,tee=True);

SCIP version 9.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.0.1] [GitHash: bebb64304e]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)

External libraries: 
  Soplex 7.0.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 1cc71921]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.6.0          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 690e9e7      AMPL .nl file reader library (github.com/ampl/mp)
  PaPILO 2.2.1         parallel presolve for integer and linear optimization (github.com/scipopt/papilo) (built with TBB) [GitHash: 3f1f0d53]
  bliss 0.77           Computing Graph Automorphisms by T. Junttila and P. Kaski (users.aalto.fi/~tjunttil/bliss)
  sassy 1.1            Sym

### Accessing Results

Syntax:

```__ResultingValue__ = pyo.value(__YourExpressionOrVariable__)```

In [15]:
totalNumShirts = pyo.value(model.N["Shirts"])
totalNumSculptures = pyo.value(model.N["Sculptures"])

print(f"There will be {totalNumShirts:.3f} Shirts made.")
print(f"There will be {totalNumSculptures:.3f} Sculptures made.")

#... This will take forever if we do it one-by-one
print("\nSchedule:")
for p in model.peopleSet:
    print(f"{p}:")
    for r in model.productSet:
        print(f"\t{r}: {pyo.value(model.H[p,r]):.3f} Hours")

There will be 25.000 Shirts made.
There will be 31.667 Sculptures made.

Schedule:
Jack:
	Sculptures: 8.000 Hours
	Shirts: 0.000 Hours
Jill:
	Sculptures: 5.222 Hours
	Shirts: 2.778 Hours


* Notice how there will be 31.667 Sculptures made. Does this make sense?
    * Perhaps you'll just have one sculpture that is incomplete. You'll finish it the next day.
    * Or maybe we want to constrain that we have to have a whole number of each product made. How would we address that?

# SECTION 2: Route Planning Problem

Here, I'll walk you through coding your first mathematical model in Pyomo.

We'll use the Route Planning Problem we've dealt with to do so.

Recall that we want to find the shortest distance between points A and D given the following paths and associated distances.

![](RoutePlanningProblem.png)

### Step 1: Define the Pyomo ConcreteModel:

I'll go ahead and do this for you since it's very simple.

In [15]:
model = pyo.ConcreteModel()

### Step 2: Define Relevant Sets:

The relevant sets for this problem are as follows.

$$\textbf{P} = \{A,B,C,D\}$$
$$\textbf{P}^{NON-TERM} = \{B,C\}$$
$$\textbf{P}^{TERM} = \{A,D\}$$
$$\textbf{R} = \{AB,AC,BC,BD,CD\}$$
$$\textbf{R}_p:$$
|$p$|$\textbf{R}_p$|
|-|-|
|$A$|$\{AB,AC\}$|
|$B$|$\{AB,BC,BD\}$|
|$C$|$\{AC,BC,CD\}$|
|$D$|$\{BD,CD\}$|

I'll go ahead and code in $R_p$. You code in the rest of the sets.

Make sure to code in each index as an individual string. For example to write the letter A or the string AB in python, you must wrap them in quotation marks: ```"A"```, ```"AB"```

In [16]:
#TODO: Code in set "P" here.


#TODO: Code in set "P_NON_TERM" here.


#TODO: Code in set "P_TERM" here.


#TODO: Code in set "R" here.


model.R_A = pyo.Set(initialize=["AB","AC"])
model.R_B = pyo.Set(initialize=["AB","BC","BD"])
model.R_C = pyo.Set(initialize=["AC","BC","CD"])
model.R_D = pyo.Set(initialize=["BD","CD"])

#This line just allows us to be able to reference each subset "p" of R_p using an easy syntax.
model.R_p = {"A": model.R_A, "B": model.R_B, "C": model.R_C, "D": model.R_D}

### Step 3: Define parameter values

For this problem, there is only one parameter: $\delta_r$

Here are the values:

|$r$|$\delta_r$|
|-|-|
|$AB$|15|
|$AC$|5|
|$BC$|4|
|$BD$|2|
|$CD$|10|

Go ahead and code in these values.

In [17]:
delta = {
    "AB": __VALUE_HERE__,
    "AC": __VALUE_HERE__,
    "BC": __VALUE_HERE__,
    "BD": __VALUE_HERE__,
    "CD": __VALUE_HERE__
}

### Step 4: Define Pyomo Variables

Here are the variables for this problem:

$$X_r \ \ \ \forall r \in \textbf{R} \ \text{(Binary Variable)}$$
$$Y_p \ \ \ \forall p \in \textbf{P} \ \text{(Binary Variable)}$$

In [18]:
#TODO: Code in the X variable here


#TODO: Code in the Y variable here


### Step 5: Define Constraints

Here are the constraints for this problem. Code them in one by one.

* If I travel into a non-terminal point, I must travel out of it.

$$\sum_{r \in \textbf{R}_p} X_r = 2Y_p \ \ \ \forall p \in \textbf{P}^{NON-TERM}$$

In [19]:
#TODO: Code in this constraint here


* If I travel into or out of a terminal constraint, I must indicate it.

$$\sum_{r \in \textbf{R}_p} X_r = Y_p \ \ \ \forall p \in \textbf{P}^{TERM}$$

In [20]:
#TODO: Code in this constraint here


* I have to visit point A and point D.
$$Y_p = 1 \ \ \ \ \forall p \in \textbf{P}^{TERM}$$

In [21]:
#TODO: Code in this constraint here


### Step 6: Define the Objective

Here is the objective function for this problem:

$$\min \sum_{r \in \textbf{R}} \delta_r X_r$$

In [22]:
#TODO: Code in this objective here


### Step 7: Solve (a.k.a. Optimize) the Model

In [23]:
#TODO: Create a solver object here (remember to use the "appsi_highs" solver)

#TODO: Solve the model here


{'Problem': [{'Lower bound': 11.0, 'Upper bound': 11.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 1}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Step 8: Extract Results.

The details of what you want to extract and how you'll represent the optimal solution are up to you and are specific to the problem you're trying to solve.

For this example problem, let's simply print out each of the paths that the solver indicates we should take.

In [24]:
#TODO: Create a Python for loop here to iterate over each of the paths in "R"


    #TODO: Extract the value of "X" for this path and store it in a python variable called "Xval"



    #TODO: Test if Xval is greater than 0.5. If it is, print out the name of this path.
    #Remember that X should return a binary value. Thus the only valid value greater than 0.5 would be 1. However, computers always have a hard time comparing numbers that are very close.
    #For example, the solver might return that the value of Xval is 0.999999999
    #If we check 0.9999999999 == 1, we will get False.
    #Thus, checking to see if Xval >= 0.5 is an acceptable way to test the value of a binary result.


AC
BC
BD


This is a simple problem that you can probably solve without a computer. Go back up and look at the map. What should the optimal path be?

Are the results you got correct?

# SECTION 3: My Recommended Pyomo Workflow

Here is another rendition of the RoutePlanning Problem but this time using my recommended way of organizing your code.

As a note, this is approach is one way to organize your code. It is by no means the only way to do it, nor is it the "best" way to do it.

But in the hundreds of models I've created, I've found this approach to be a good way to keep things organized, readable, and typo-free.

In [25]:
class Parameters:
    """
    A class to house all the parameters relevant to the Route Planning Example Problem
    """
    def __init__(self):
        #Specify all parameters here making sure to say "self." before each one.
        self.Points = ["A","B","C","D"]
        self.Connections = ["AB","AC","BC","BD","CD"]
        self.TerminalPoints = ["A","D"]

        self.delta = {
            "AB": 15,
            "AC": 5,
            "BC": 4,
            "BD": 2,
            "CD": 10
        }

In [26]:
from PyomoTools import LoadIndexedSet

def AssembleModel(params:Parameters):
    """
    A function to generate an instance of the Pyomo model for the Route Planning Example Problem.

    Parameters
    ----------
    params: Parameters
        The Parameters object containing the parameters relevant to this instance.

    Returns
    -------
    model: pyo.ConcreteModel
        The Pyomo model for this instance.
    """

    ### STEP 1: Define the Pyomo model
    model = pyo.ConcreteModel()

    ### STEP 2: Define sets
    model.P = pyo.Set(initialize=params.Points)
    model.P_NON_TERM = pyo.Set(initialize=list(set(params.Points) - set(params.TerminalPoints)))
    model.P_TERM = pyo.Set(initialize=params.TerminalPoints)
    model.R = pyo.Set(initialize=params.Connections)
    
    Rp = {p: [r for r in model.R if p in r] for p in model.P}
    LoadIndexedSet(model,"R_p",Rp) #This is a function I created to automate the process of adding subsets into a Pyomo model.

    ### Step 3: Define oarameters
    #This is already done in the Parameters class

    ### Step 4: Define variables
    model.X = pyo.Var(model.R,domain=pyo.Binary)

    model.Y = pyo.Var(model.P,domain=pyo.Binary)

    ### Step 5: Define constraints
    def NonTerminalTravelConstraint(model,p):
        return sum(model.X[r] for r in model.R_p[p]) == 2 * model.Y[p]
    model.NonTerminalTravelConstraint = pyo.Constraint(model.P_NON_TERM,rule=NonTerminalTravelConstraint)

    def TerminalTravelConstraint(model,p):
        return sum(model.X[r] for r in model.R_p[p]) == model.Y[p]
    model.TerminalTravelConstraint = pyo.Constraint(model.P_TERM,rule=TerminalTravelConstraint)

    def TerminalMandate(model,p):
        return model.Y[p] == 1
    model.TerminalMandate = pyo.Constraint(model.P_TERM,rule=TerminalMandate)

    ### Step 6: Define Objective
    model.Obj = pyo.Objective(expr=sum(params.delta[r] * model.X[r] for r in model.R))

    return model


In [27]:
def ExecuteOptimization(model:pyo.ConcreteModel):
    """
    A function to execute the optimization of a pyomo model.

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model you'd like to optimize

    Returns
    -------
    None
    """

    #Sometimes there are lots of settings you want to set or special functions you want to call to record the output of this "solve call". That's why I normally have a dedicated function for this. But I suppose it's less necessary for this simple problem.
    solver = pyo.SolverFactory("appsi_highs")
    solver.solve(model)

In [29]:
def ExtractResults(model:pyo.ConcreteModel):
    """
    A function to extract the results of a Route Planning Problem model

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model from which you'd like to extract the results

    Returns
    -------
    traveledPaths: list
        A list of the paths that are to be traveled under the optimal solution
    """
    traveledPaths = []
    for r in model.R:
        Xval = pyo.value(model.X[r])
        if Xval >= 0.5:
            traveledPaths.append(r)
    return traveledPaths

In [30]:
def main():
    params = Parameters()
    model = AssembleModel(params)
    ExecuteOptimization(model)
    results = ExtractResults(model)

    print(results)

main()

['AC', 'BC', 'BD']
